<img src="https://egc2023.sciencesconf.org/data/pages/logo_2.jpg" alt="EGC 2023" width="200px"/>


# Démonstration - Perdido Geoparser - EGC 2023 

Cette démonstration s'appuie sur le tutoriel présenté lors de l'atelier [Librairies Python et Services Web pour la reconnaissance d’entités nommées et la résolution de toponymes](https://anf-tdm-2022.sciencesconf.org/resource/page/id/11) de la formation CNRS [ANF TDM 2022](https://anf-tdm-2022.sciencesconf.org).
Elle présente la librairie Python [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding de textes en français. 


**Auteurs** : [Ludovic Moncla](https://ludovicmoncla.github.io) (Univ Lyon, INSA Lyon, CNRS, UCBL, LIRIS, UMR 5205, F-69621)
[Mauro Gaio](https://lma-umr5142.univ-pau.fr/fr/organisation/membres/cv_-mgaio-fr.html) (Université de Pau et des Pays de l'Adour, CNRS, LMAP, UMR 5142)

## 1. En bref


Dans cette démonstration, nous allons voir comment :

- Charger des jeux de données :
  - à partir de fichiers txt importés depuis le disque dur ;
  - à partir de la librairie Python [Perdido](https://github.com/ludovicmoncla/perdido) dans un [Pandas dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (articles encyclopédiques et descriptions de randonnées).
- Manipuler et interroger un dataframe
- Utiliser la librarie [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding :
  - afficher les entités nommées annotées ;
  - cartographier les lieux geocodés ;
  - illustrer la problématique de désambiguïsation des toponymes.
  - enregistrer les résultats dans différents formats (csv, dataframe, ...)

## 2. Configurer l'environnement


* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la 2.2.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter les cellules suivantes :

### 2.1 Installer les librairies Python (uniquement si vous n'avez pas configuré l'environnement Python)

In [ ]:
! pip install perdido==0.1.29
! pip install stanza==1.4.2

### 2.2 Importer les librairies


Tout d'abord, nous allons charger certaines bibliothèques spécifiques de `Perdido` que nous utiliserons dans ce notebook. Ensuite, nous importons quelques outils qui nous aideront à analyser et à visualiser le texte.

In [1]:
import warnings
warnings.filterwarnings('ignore')

from perdido.geoparser import Geoparser
from perdido.geocoder import Geocoder

from perdido.datasets import load_edda_artfl, load_edda_perdido, load_choucas_perdido

from spacy import displacy

## 3. Chargement et exploration des données

### 3.1 Chargement d'un document texte à partir d'un fichier


In [ ]:
# On définit une fonction qui prend en paramètre le chemin d'un fichier et qui retourne sont contenu

def load_txt(filepath):
    with open(filepath) as f:
        return f.read()

In [ ]:
# On utilise la fonction précédente pour récupérer le contenu de l'article encyclopédique 'Arques' (volume01-4083.txt) présent dans le dossier data
arques = load_txt('data/edda-volume01-4083.txt')

* Afficher le contenu du fichier

In [ ]:
print(arques)

### 3.2 Chargement d'un jeu de données à partir de la librairie Perdido

La libraire de geoparsing [Perdido](https://github.com/ludovicmoncla/perdido) embarque deux jeux de données : 
 1. des articles encyclopédiques (volume 7 de l'Encyclopédie de Diderot et d'Alembert (1751-1772)), fournit par l'[ARTFL](https://encyclopedie.uchicago.edu) dans le cadre du projet [GEODE](https://geode-project.github.io) ;
 2. des descriptions de randonnées (chaque description est associée à sa trace GPS. Elles proviennent du site [www.visorando.fr](https://www.visorando.com) et ont été collectées dans le cadre du projet [ANR CHOUCAS](http://choucas.ign.fr).

 Dans un premier temps nous allons nous intéresser au jeu de données des articles encyclopédiques. Ce jeu de données est présent dans la librairie en deux versions, une version "brute" (articles fournis par l'ARTFL) au format dataframe et une version déjà annotée par Perdido (format PerdidoCollection). Nous allons charger la version brute et voir comment manipuler un dataframe.

* Charger le jeu de données :

In [2]:
dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']

* Afficher les informations sur le jeu de données :

In [3]:
data_artfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3385 entries, 0 to 3384
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filename   3385 non-null   object
 1   volume     3385 non-null   int64 
 2   number     3385 non-null   int64 
 3   head       3384 non-null   object
 4   normClass  3384 non-null   object
 5   author     3384 non-null   object
 6   text       3385 non-null   object
dtypes: int64(2), object(5)
memory usage: 185.2+ KB


On remarque que certaines colonnes ont une donnée manquante (3384 lignes non nulles contre 3385 lignes au total). Pour la suite des opérations que nous allons réaliser il est nécessaire de supprimer les lignes incomplètes.

* Supprimer la ligne incomplète :

In [ ]:
data_artfl.dropna(inplace=True)     # data_artfl = data_artfl.dropna()

* Vérifier le résultat :

In [ ]:
data_artfl.info()

* Afficher la liste des premiers articles :

In [ ]:
data_artfl.head()

### 3.3 Manipulation d'un dataframe

Nous avons maintenant accès à tous les attributs et méthodes de l'objet [dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). Par exemple, nous pouvons facilement connaître le nombre de lignes dans notre dataframe qui correspond au nombre d'articles dans notre corpus :

In [ ]:
n = data_artfl.shape[0]
print('Il y a ' + str(n) + ' articles dans le jeu de données.')

#### 3.3.1 Recherche par métadonnées


Maintenant que les données sont chargées dans un dataframe, nous pouvons sélectionner des groupes d'articles sur la base de leurs métadonnées.

Pour cela on utilise la méthode [loc()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html).



* Selectionner la ligne du dataframe qui correspond à l'article 'FRONTIGNAN' :

In [ ]:
frontignan = data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN']
frontignan

* Récupérer les valeurs des attributs (colonnes) :

In [ ]:
print('volume :', frontignan.volume.item())   # similaire à frontignan['volume'].item()
print('number :', frontignan.number.item())
print('text :', frontignan.text.item())

Nous pouvons également filtrer les données sur la base de l'auteur.

* Extraire les articles rédigés par Jaucourt :

In [ ]:
req = 'Jaucourt'
d_Jaucourt = data_artfl.loc[data_artfl['author'] == req]

n = d_Jaucourt.shape[0]
print(str(n) + ' articles ont été rédigés par '+ req)

* Afficher les 5 premiers :

In [ ]:
d_Jaucourt.head()

### 3.3.2 Recherche par mots-clés

Autre exemple, nous pouvons filtrer les articles en fonction de leur classification dans l'*Encyclopédie*. 
Pour cela nous utiliserons la colonne `normclass`, qui indique la classifications retenue (et normalisée) par l'ARTFL. 

Par exemple pour la classe 'Géographie', nous pouvons faire la requête suivante (le résultat est stocké dans un nouveau dataframe `d_geo` : 

In [ ]:
req = 'Géographie'
d_geo = data_artfl[data_artfl['normClass'].str.contains(req, case=False)]

n = d_geo.shape[0]
print(str(n) + ' articles sont classés en '+ req)

In [ ]:
req = 'ville de'
d_geo = data_artfl[data_artfl['text'].str.contains(req, case=False)]

n = d_geo.shape[0]
print(str(n) + " articles contiennent l'expression '"+ req + "'")

### 3.3.3 Regroupements

On peut également regrouper les données selon un ou plusieurs attributs (colonnes) et compter le nombre de données de chaque groupe avec les méthodes [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) et [count()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html).

* Afficher le nombre d'articles classés en Géographie par auteur :

In [ ]:
d_geo.groupby(['author'])["filename"].count()

Dans cette partie nous avons vu brievement comment manipuler un dataframe pour selectionner certaines données en filtrant selon certaines métadonnées ou par une recheche par mot clés. Ces opérations sont utiles mais un peu limitées, nous allons voir dans la suite de ce notebook comment enrichir les métadonnées et en particulier comment annoter les entités nommées présents dans les textes.

## 4. Reconnaissance d'Entités Nommées (NER)

La reconnaissance d'entités nommées, *Named Entity Recognition* (NER) en anglais, est une tâche très importante et incontournable en traitement automatique des langues (TAL) et en compréhension du langage naturel (NLU en anglais). 
Cette tâche consiste à rechercher des objets textuels (un mot, ou un groupe de mots, souvent associés aux noms propres) catégorisables dans des classes telles que noms de personnes, noms d'organisations ou d'entreprises, noms de lieux, quantités, distances, valeurs, dates, etc.
Les typologies et les jeux d'étiquettes sont dépendents de chaque outil.
Dans le cas de Perdido ...

### 4.3 Perdido Geoparser


`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le geotagging et le geocoding. Le geotagging est similaire à la tâche de reconnaissance des entités nommées avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).
Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques non ambigues.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour la reconnaissance d'entités nommées toujours à partir de notre exemple `Arques`.

* Instancier et paramétrer la chaîne de traitement :

In [ ]:
geoparser = Geoparser(version="Encyclopedie")

* Executer la reconnaissance d'entités nommées :

In [ ]:
arques_perdido = geoparser(arques)

Perdido effectuant la tâche de geocoding en plus du NER, le temps de traitement est plus long qu'avec Stanza ou spaCy, du fait de l'interrogation de ressources geographiques externes pour chaque nom de lieu repéré.

* Afficher la liste des entités nommées repérées :

In [ ]:
for ent in arques_perdido.named_entities:
    print(ent.text, ent.tag)

* Afficher de manière graphique les entités nommées avec `displaCy` grâce à la méthode de conversion `to_spacy_doc`:

In [ ]:
displacy.render(arques_perdido.to_spacy_doc(), style="ent", jupyter=True)

* Un rendu similaire mais qui permet de visualiser les entités imbriquées (`style="ent"` -> `style="span"`) :

In [ ]:
displacy.render(arques_perdido.to_spacy_doc(), style="span", jupyter=True)

Cet exemple permet d'illustrer les différences qu'il peut y avoir entre des outils de NER généraliste et ou un outil de geoparsing. On observe ici que Perdido permet une annotation plus fine grâce aux entités imbriquées (ville de, petite rivière) ainsi que le repérage des coordonnées géographiques. En fonction du besoin le repérage de ces éléments peut etre utile pour les traitements suivants ou les analyses qui s'appuient sur ces résultats. 

### 4.4 Expérimentations et comparaison

* Charger l'article `Beaufort` (volume 2, numéro 1365) disponible dans le dossier `data` :

In [ ]:
beaufort = load_txt('data/edda-volume02-1365.txt')

print(beaufort)

* Perdido

In [ ]:
beaufort_perdido = geoparser(beaufort)
displacy.render(beaufort_perdido.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

In [ ]:
lge_beaufort = load_txt('data/lge-beaufort.txt')
print(lge_beaufort)

Cette fois l'article est un peu plus long et comporte des césures de lignes importantes, définissons donc une fonction pour recoller les morceaux :

In [ ]:
def join_lines(s):
    return s.replace('¬\n', '').replace('-\n', '').replace('\n', ' ')

In [ ]:
normalized_lge_beaufort = join_lines(lge_beaufort)
normalized_lge_beaufort

* Perdido

In [ ]:
lge_beaufort_perdido = geoparser(normalized_lge_beaufort)
displacy.render(lge_beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

## 5. Geoparsing / Geocoding

En complément de la tâche de reconnaissance des entités nommées la librairie `Perdido` propose également celle de résolution des toponymes, on parle alors de *Geoparsing*. Cette tâche consiste a associer à un nom de lieu des coordonnées géographiques non ambigus. De manière classique elle s'appuie sur le repérage des entités spatiales identifées lors de la reconnaissance des entités nommées et fait appel à des ressources externes de type *gazetier* (ou dictionnaires topographique) pour localiser les lieux.

### 5.1 Perdido Geoparser

* Revenons à l'article `ARQUES`

In [ ]:
print(arques)
displacy.render(arques_perdido.to_spacy_doc(), style="ent", jupyter=True)

* En plus de pouvoir afficher la liste des entités nommées comme nous l'avons fait précédemmment, nous pouvons directement afficher la carte des lieux localisés

In [ ]:
# afficher la carte des lieux localisés
arques_perdido.get_folium_map()

Par défaut, lors de l'instanciation du `Geoparser()`, seul [OpenStreetMap](https://www.openstreetmap.org/) est utilisé pour le geocoding et au maximum un résultat est retourné pour chaque lieu (nous verrons dans la suite comment paramétrer le geocoding).

On a déjà ici un aperçu de la difficulté de la tâche de résolution des toponymes. En effet, un grand nombre d'ambiguïtés existent tels que plusieurs lieux ayant le même nom, plusieurs noms pour un même lieu ou encore le fait qu'un lieu ne soit pas référencé dans les ressources que l'on interroge.

### 5.2 Perdido Geocoder

En complément du `Geoparser` qui prend en paramètre un texte et qui fait la reconnaissance d'entités nommées en amont de l'étape de geocoding, `Perdido` propose également une fonction de geocoding disctincte prenant en paramètre directement un nom de lieu (ou une liste de noms de lieux).

In [ ]:
geocoder = Geocoder()
doc = geocoder(['Arques', 'France', 'Normandie', 'Caux'])

# afficher la carte des lieux localisés
doc.get_folium_map()

### 5.3 Résolution de toponymes / désambiguïsation


#### 5.3.1 Exemple : Arques

* Cherchons à localiser la ville `Arques`


In [ ]:
geocoder = Geocoder()
doc = geocoder('Arques')
doc.get_folium_map()

On remarque que par défaut, la localisation retournée pour le nom de lieu `Arques` n'est pas celle que l'on recherche. En effet, le texte indique qu'il s'agit d'une ville de Normandie, or ici la localisation proposée est située dans le Pas-de-Calais !

Changeons les paramètres du `Geocoder` (ces paramètres sont similaires pour le `Geoparser`) pour essayer de retrouver la bonne localisation.

* Augmenter le nombre de résultats retournés par les gazetiers interrogés

In [ ]:
geocoder = Geocoder(max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

On observe parmi les 10 localisations retournées par OpenStreetMap (gazetier par défaut) qu'aucune ne se situe en Normandie.

* Remplacer OpenStreetMap par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'])
doc = geocoder('Arques')
doc.get_folium_map()

On observe que le premier résultat retourné par l'IGN ne se situe ni en Normandie (comme attendu), ni dans le Pas-de-Calais comme le premier résultat retourné par OpenStreetMap.

* Augmenter le nombre de résultats retournés par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'], max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

Cette fois-ci on retrouve bien une localisation en Normandie au sud de Dieppe avec pour nom `Arques-la-Bataille'. On peut faire l'hypotèse que le nom a évolué car cette localisation se situe bien dans le Pays de Caux (voir illustration ci-dessous, source [Wikipedia](https://fr.wikipedia.org/wiki/Pays_de_Caux)) comme l'indique le texte de l'article.

![Pays de Caux](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Carte_pays_Caux1.png/497px-Carte_pays_Caux1.png)

Ce problème d'ambiguïtés peut aussi être illustrer en allant directement interroger le site web du [géoportail](https://www.geoportail.gouv.fr) comme le montre la capture ci-dessous.

![Résultats sur le géoportail](https://github.com/lmoncla/demo-perdido-egc-2023/blob/main/img/geoportail-arques.png)


Il reste néanmoins le problème de retrouver cette localisation de manière automatique. 
Plusieurs approches existent dans la littérature mais ne sont pas encore implémentées dans `Perdido`.

Cet exemple illustre bien la difficulté de la problématique de désambiguïsation des toponymes avec notamment la gestion des natures de lieux différentes (pays, régions, communes, lieux-dits, lac, rivières, etc.) associés à un même nom, l'homonymie, la non exaustivité des ressources, l'évolution des noms au cours du temps ou encore les erreurs d'orthographe.

* Afficher la carte obtenue après le geoparsing avec l'IGN et 10 résultats max par nom de lieu

In [ ]:
geoparser = Geoparser(sources=['ign'], max_rows=10)
doc = geoparser(arques)
doc.get_folium_map()


### 5.4 Le cas des descriptions de randonnées

Prenons maintenant l'exemple du geoparsing de descriptions de randonnées. Certaines solutions de désambiguisation ont pu être développées et intégrées au sein de la librairie `Perdido` (d'autres sont en cours d'intégration). Les solutions décrites dans la suite de cette partie ont été développées dans le cadre des projets [Perdido](http://erig.univ-pau.fr/PERDIDO/) (2012-2015) et [ANR CHOUCAS](http://choucas.ign.fr) (2017-2022). 

> Ludovic Moncla, Walter Renteria-Agualimpia, Javier Nogueras-Iso and Mauro Gaio (2014). "Geocoding for texts with fine-grain toponyms: an experiment on a geoparsed hiking descriptions corpus". In Proceedings of the 22nd ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, pp 183-192.

> Mauro Gaio and Ludovic Moncla (2019). “Geoparsing and geocoding places in a dynamic space context.“ In The Semantics of Dynamic Space in French: Descriptive, experimental and formal studies on motion expression, 66, 353.


Nous avons choisi un exemple pour illustrer les différentes phases du processus que nous avons mis en place dans le cadre du geoparsing de descriptions de randonnées :
1. filtrer les résultats en fonction du pays 
2. filtrer les résultats en fonction d'une zone géographique définie 
3. regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*)
4. selectionner le cluster qui contient le plus d'entités distinctes

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 
Cette stratégie est adaptée pour une description d'itinéraire où les différents lieux cités sont supposés être localisés à proximité les uns des autres.

* Charger le jeu de données CHOUCAS de descriptions de randonnées fourni par `Perdido`

In [4]:
dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

data_choucas.to_dataframe().head()

,name,text,geometry,#_places,#_person,#_event,#_date,#_misc,#_locations
0,Chalets de la Fullie,\n\nBoucle des chalets de la Fullie au départ ...,"(LINESTRING (6.11174 45.616041, 6.11174 45.616...",17,0,0,0,0,17
1,Traversée cabane de Pravouta à la Plagne,\n\nPartir de la cabane de Pravouta juste de l...,"(LINESTRING (5.832543 45.315222, 5.832444 45.3...",23,2,0,0,0,23
2,Refuge Entre Le Lac - Refuge de la Leisse,\n\nDépart du refuge d'Entre le Lac près du la...,"(LINESTRING (6.839184 45.480323, 6.83987 45.47...",22,0,0,0,0,22
3,Le lac du Retour,"\n\nDu parking de Pierre Giret, suivre la rout...","(LINESTRING (6.917631 45.619278, 6.917527 45.6...",6,1,0,0,0,6
4,Traversée Alpette - Dent de Crolles,\n,"(LINESTRING (5.907402 45.440585, 5.907439 45.4...",0,0,0,0,0,0


In [5]:
len(data_choucas)

30

* Sélectionner une randonnée (parmi les 30)

In [6]:
id_rando = 2
doc = data_choucas[id_rando]

In [7]:
doc.text

"\n\nDépart du refuge d'Entre le Lac près du lac de la Plagne.\nDu refuge Entre le Lac, un sentier remonte les pentes herbeuses et permet de rejoindre le GR5 un peu avant le chalet de la Grassaz (chalet du berger 2335m). Toujours en direction du sud, on remonte le vallon en longeant le ruisseau. On parvient ainsi à l'extrémité ouest du lac de Grattaleu; un peu plus haut, on atteint le refuge du col du Palet (2550m). On admire la beauté de la vallée et le sommet de Bellecote recouvert de glaciers. Le GR descend vers l'Est; le sentier serpente entre des entonnoirs créés dans le gypse par dissolution. Le GR passe sous un 1er télésiège, celui de Grattaleu, et près de l'arrivée d'un second, le Tichot. Au chalet de Lognan (croix) prendre à droite un sentier qui descend à Val Claret (2107m) (station de ski). Poursuivre jusqu'au chalet de la Leisse. Le GR55 s'élève vers le vallon du paquis. On passe en contrebas du chalet du Prariond; un peu plus loin on arrive à la bifurcation du col de Fress

In [8]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [9]:
doc.get_folium_map()

On observe ici le résultat déjà pré-traité par `Perdido`. Nous allons maintenant illustrer le processus de désambiguïsation.

On recommence le processus de geoparsing en entier à partir du texte de la randonnées choisie.

In [ ]:
geoparser = Geoparser()
doc_geoparsed = geoparser(doc.text)

In [ ]:
doc_geoparsed.get_folium_map()

On voit clairement la différence par rapport au résultat précédent. Nous allons alors essayer de retrouver le même résultat en déroulant les différentes étapes pour désambiguïser avec `Perdido`.

Pour gagner un peu de temps lors des prochaines executions nous allons faire directement appel à la fonction de geocoding à partir de la liste des noms de lieux.

* Récuperer la liste des noms de lieux (sans doublon)

In [ ]:
places_list = list(set([ent.text for ent in doc_geoparsed.ne_place]))
print(places_list)

#### 5.4.1 Ajout d'un filtre "code pays"


In [ ]:
# instancier le geocoder avec le code pays
geocoder = Geocoder(country_code = 'fr')
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

doc_geocoded.get_folium_map()

#### 5.4.2 Ajout d'un filtre "bounding box"

In [ ]:
bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403] # zone d'intervention du PGHM Isère

# instancier le geocoder avec le code pays et une bounding box
geocoder = Geocoder(country_code = 'fr', bbox = bbox)
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

# affiche la carte
doc_geocoded.get_folium_map()

#### 5.4.3 Clustering par densité spatiale

In [ ]:
# appliquer la désambiguïsation 
doc_geocoded.cluster_disambiguation()
doc_geocoded.get_folium_map()